In [2]:
!pip install casadi

In [4]:
import numpy as np
import casadi as cas
from casadi import SX

In [5]:
import plotly
#print(plotly.__version__)
import plotly.graph_objs as go
from ipywidgets import widgets

In [6]:
n_brackets = 4
tax0 = -5
bracket = np.array([0, 10, 20, 50, 100])   # bracket boundaries

In [7]:
t = np.array([0, 1/10, 6/10, 1/10])

In [8]:
taxpt = np.zeros(n_brackets)
taxpt[0] = tax0
for i in range(1, n_brackets):
    taxpt[i] = taxpt[i-1] + t[i-1] * (bracket[i] - bracket[i - 1])

In [9]:
def taxes(bracket, t, taxpt, yinc):
    
    u_bound_index = np.argmin(bracket <= yinc)
    l_bound_index = np.argmax(bracket > yinc) - 1 
    
    res = taxpt[l_bound_index] + t[l_bound_index] * (yinc - bracket[l_bound_index])
    return res
    

In [10]:
taxes(bracket, t, taxpt, [10])

array([-5.])

In [11]:
y_incomes = np.arange(80)
results = np.array([taxes(bracket, t, taxpt, income) for income in y_incomes])

tax_plot = go.FigureWidget(data=
    [dict(x = y_incomes,
    y = results, type='scatter') ]
)
tax_plot.layout.title = 'Tax (income)'
income_plot = go.FigureWidget(data=
    [dict(x = y_incomes,
    y = y_incomes - results, type='scatter') ]
)
income_plot.layout.title = 'Income - tax (income)'
widgets.HBox([tax_plot, income_plot])

    'data': [{'type': 'scatter',
              'uid': 'f09bc698-7365-4d2f-9471-4…

In [12]:
tax_plot = go.FigureWidget(data=
    [dict(x = y_incomes,
    y = results, type='scatter') ]
)
tax_plot.layout.title = 'Tax (income)'
income_plot = go.FigureWidget(data=
    [dict(x = y_incomes,
    y = y_incomes - results, type='scatter') ]
)
income_plot.layout.title = 'Income - tax (income)'
widgets.HBox([tax_plot, income_plot])

    'data': [{'type': 'scatter',
              'uid': '4f382d2e-dd71-48c8-8903-3…

In [13]:
def utility(consumption, consumption_b, labour, labour_b, eta, gamma):
    B = 1/10
    util = 100 * ((consumption-consumption_b)**(1-gamma)/(1-gamma) - 
                  B*(labour+labour_b)**(1+eta)/(1+eta) + 1)
    # WHY + 1????
    return util

In [14]:
# define casadi type symbolic variables
consumption = SX.sym('consumption', n_brackets, 1)
labour = SX.sym('labour', n_brackets, 1)
p = SX.sym('p', n_brackets, 1)
yincs = SX.sym('yincs', n_brackets, 1)


In [15]:
eta = 1
gamma=2
wage = 100
consumption_b = 1
labour_b = 0

In [16]:
# define objective function
objective = (p.T @ utility(consumption, consumption_b, labour, labour_b, eta, gamma))

In [17]:
## define bounds
#  lower bounds
c_lb = np.full(n_brackets, 0.0001 + consumption_b)
l_lb = np.full(n_brackets, 0.0001 + labour_b)
p_lb = np.full(n_brackets, 0.25 )
yin_lb = bracket[0:4]


#  upper bounds

c_ub = np.full(n_brackets, np.inf)
l_ub = np.full(n_brackets, np.inf)
p_ub = np.full(n_brackets, 0.25 )
#p_ub = np.ones(n_brackets)
yin_ub = bracket[1:5]

In [18]:
bracket[0:4]

array([ 0, 10, 20, 50])

In [19]:
def budget_constraint(bracket, yinc, t, n_brackets, consumption):
    constraints = cas.SX.zeros(n_brackets, 1)
    for i in range(n_brackets):
        temp_con = -tax0 + yinc[i] - t[i] * (yinc[i] - bracket[i]) 
        for j in range(i):
            temp_con = temp_con - (bracket[i-j] - bracket[i-j-1]) * t[i-j-1]
        constraints[i] = (temp_con)
    constraints = constraints - consumption 
    return constraints
    #cons_1 = - tax0  + yinc[0] - t[0] * (yinc[0] - bracket[0])
    #cons_2 = - tax0  + yinc[1] - (bracket[1] - bracket[0]) * t[0] \
    #                 - t[1] * (yinc[1] - bracket[1])
    #cons_2 = - tax0  + yinc[1] - (bracket[1] - bracket[0]) * t[0] \
    #                 - t[1] * (yinc[1] - bracket[1])
    

In [20]:
def asset_constraint(yinc, labour, wage):
    constraints = wage * labour - yinc

    return constraints
    #cons_1 = - tax0  + yinc[0] - t[0] * (yinc[0] - bracket[0])
    #cons_2 = - tax0  + yinc[1] - (bracket[1] - bracket[0]) * t[0] \
    #                 - t[1] * (yinc[1] - bracket[1])
    #cons_2 = - tax0  + yinc[1] - (bracket[1] - bracket[0]) * t[0] \
    #                 - t[1] * (yinc[1] - bracket[1])
    

In [21]:
# probability constraint 
p_constraint = cas.sum1(p) - 1

In [22]:
## define constraints
# Budget constraint

budget_constraint(bracket, yincs, t, n_brackets, consumption)
#asset_constraint(yinc, labour, wage)
#p_constraint

SX(@1=5, @2=0.1, @3=1, [((@1+yincs_0)-consumption_0), (((@1+yincs_1)-(@2*(yincs_1-10)))-consumption_1), ((((@1+yincs_2)-(0.6*(yincs_2-20)))-@3)-consumption_2), (((((@1+yincs_3)-(@2*(yincs_3-50)))-18)-@3)-consumption_3)])

In [23]:
consumption

SX([consumption_0, consumption_1, consumption_2, consumption_3])

In [24]:
# penalty for case that both are similar
penalty = 0.001 * (p @ p.T)

In [25]:
penalty = 0.001*(sum([p[i]*p[j]  for i in range(n_brackets - 1) 
                for j in range(i + 1, n_brackets)]) 
           + p[1] + 2 * p[2] + 3 * p[3])

In [26]:
#penalty = 0 * p[0]
#for i in range(n_brackets - 1):
#    for j in range(i + 1, n_brackets):
#        penalty = penalty + p[i]*p[j]     

#penalty = penalty + p[1] + 2 * p[2] + 3 * p[3]

In [27]:
opt_objpen = objective - penalty

In [28]:
penalty

SX((0.001*(((((((((p_0*p_1)+(p_0*p_2))+(p_0*p_3))+(p_1*p_2))+(p_1*p_3))+(p_2*p_3))+p_1)+(2*p_2))+(3*p_3))))

In [29]:
# NLP definition = Our optimization problem
nlp = {'x': cas.vertcat(consumption, labour, p, yincs),
       'f': -opt_objpen,
       'g': cas.vertcat(budget_constraint(bracket, yincs, t, n_brackets, consumption),
                        asset_constraint(yincs, labour, wage),
                        p_constraint)
      }

#  solver_opt = {'verbose': False, 'print_time': False, 'ipopt': {'print_level': 5,
#                                                               'tol': 1E-12, 'constr_viol_tol': 1E-12}}
#  solver_opt = dict(qpsol='qrqp', qpsol_options=dict(print_iter=False, tol=1e-16), print_time=False)

# creating casadi optimization object
#solver_opt = dict(qpsol='qrqp', hessian_approximation='exact', min_step_size=0,
#                  print_header=False, print_iteration=True, print_status=True,
#                  qpsol_options=dict(print_iter=False, jit=True, tol=1E-10), print_time=False, max_iter=200)
#S = cas.nlpsol("S", "sqpmethod", nlp, solver_opt)
solver_opt = {'ipopt': {'print_level': 5,'tol': 1E-11, 'constr_viol_tol': 1E-11}}

S = cas.nlpsol("S", "ipopt", nlp, solver_opt)


In [30]:
res = S(x0=100, lbg=0, ubg=0, lbx=np.concatenate([c_lb, l_lb, p_lb, yin_lb]), ubx=np.concatenate([c_ub, l_ub, p_ub, yin_ub]))


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       16
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        8

Total number of variables............................:       12
                     variables with only lower bounds:        8
                variables with lower and upper bounds:        4
                     variables with only upper bounds:        0
Total number of equa

In [31]:
res['x']

DM([15, 24, 32.3155, 41.1778, 0.1, 0.2, 0.407888, 0.557532, 0.25, 0.25, 0.25, 0.25, 10, 20, 40.7888, 55.7532])

In [32]:
result = res['x'].full()
result

array([[15.0000001 ],
       [24.00000018],
       [32.31551646],
       [41.17783626],
       [ 0.1       ],
       [ 0.2       ],
       [ 0.40788791],
       [ 0.55753151],
       [ 0.25      ],
       [ 0.25      ],
       [ 0.25      ],
       [ 0.25      ],
       [10.        ],
       [20.        ],
       [40.78879114],
       [55.7531514 ]])

In [33]:
consumption_r = result[0:4]
labour_r = result[4:8]
yincs_r = result[-4:]

for i in range(4):
    print(utility(consumption_r[i], consumption_b, labour_r[i], labour_b, eta, gamma))


[92.80714291]
[95.45217394]
[95.97483211]
[95.95685862]


In [34]:
labour_r *100


array([[10.0000001 ],
       [20.0000002 ],
       [40.78879114],
       [55.7531514 ]])

In [35]:
yincs_r

array([[10.        ],
       [20.        ],
       [40.78879114],
       [55.7531514 ]])